In [3]:
import supervisely as sly

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
project = sly.read_project("../data/sly_dataset")
meta = project.meta
ds = project.datasets.get('ds1')

# TODO: make selected_classes as input
selected_classes = [obj_class.name for obj_class in meta.obj_classes]
cat2id = {name: i + 1 for i, name in enumerate(selected_classes)}
img_id = 1
ann_id = 1
coco_anno = {"images": [], "categories": [], "annotations": []}
for name in ds.get_items_names():
    ann = ds.get_ann(name, meta)
    img_dict = {
        "id": img_id,
        "height": ann.img_size[0],
        "width": ann.img_size[1],
        "file_name": name,
    }
    coco_anno["images"].append(img_dict)
    
    for label in ann.labels:
        if isinstance(label.geometry, (sly.Bitmap, sly.Polygon)):
            rect = label.geometry.to_bbox()
        elif isinstance(label.geometry, sly.Rectangle):
            rect = label.geometry
        else:
            pass
        x,y,x2,y2 = rect.left, rect.top, rect.right, rect.bottom
        ann_dict = {
            "id": ann_id,
            "image_id": img_id,
            "category_id": cat2id[label.obj_class.name],
            "bbox": [x, y, x2 - x, y2 - y],
            "area": (x2 - x) * (y2 - y),
            "iscrowd": 0
        }
        coco_anno["annotations"].append(ann_dict)
        ann_id += 1
    
    img_id += 1

coco_anno["categories"] = [{"id": i, "name": name} for name, i in cat2id.items()]

In [40]:
from pycocotools.coco import COCO

coco_api = COCO()
coco_api.dataset = coco_anno
coco_api.createIndex()

creating index...
index created!


In [42]:
# Save to file
import json
with open('coco_anno.json', 'w') as f:
    json.dump(coco_anno, f)

In [54]:
from src.data.coco import CocoDetection
transforms = None
ds = CocoDetection("../data/sly_dataset/ds1/img", "../data/sly_dataset/ds1/coco_anno.json", transforms, return_masks=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [1]:
import os
from src.solver import DetSolver
from src.core import YAMLConfig

config_path = 'configs/rtdetr/rtdetr_r18vd_6x_coco.yml'
checkpoint_url = "https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetr_r18vd_dec3_6x_coco_from_paddle.pth"

import torch
torch.hub.load_state_dict_from_url(checkpoint_url, "../models");

name = os.path.basename(checkpoint_url)
checkpoint_path = f"../models/{name}"

In [4]:
cfg = YAMLConfig(
    config_path,
    # resume=None,
    # use_amp=args.amp,
    tuning=checkpoint_path
)

# cfg.device = torch.device("cpu")
solver = DetSolver(cfg)

In [1]:
solver.fit()

NameError: name 'solver' is not defined